In [ ]:
# Instalo lo necesario
! python -m pip install py2neo neo4jupyter
import requests
from datetime import datetime
import json
import sys
import py2neo
from py2neo import Graph, Node, Relationship
import neo4jupyter
neo4jupyter.init_notebook_mode()

<IPython.core.display.Javascript object>

In [ ]:
# url de la BD
dburl='neo4j+s://053b5ce0.databases.neo4j.io'

# usuario y pass a la BD
user='neo4j'
pasw='FU9eXzRPH16jq7VblzQfIEJMe-NgP4WW8nWs-dFm4jM'

# conexión a la BD
graph = Graph(dburl, auth=(user, pasw))

In [ ]:
# Función que crea el nodo de un evento si no existe y si ya existe retorna el nodo existente.
def crear_evento(graph, net, code, ids, tipo, title, place, mag, time, url, detail, latitud, longitud, z):
  try:
    # Consulta si ya existe un nodo "evento" con el id dado
    query = "MATCH (e:Evento) WHERE e.net = $net and e.code = $code RETURN e"
    result = graph.run(query, net=net, code=code).data()

    if not result:
        # El nodo no existe, así que lo creamos
        print('Creando evento: ',net, code, '(', latitud, ',', longitud, ',', z,')')
        location = py2neo.data.spatial.CartesianPoint((latitud, longitud, z))
        event_node = Node("Evento",net=net, code=code, ids=ids, coordenadas=location, magnitud=mag, fecha=time, web=url, tipo=tipo, titulo=title, lugar=place, detalle=detail)
        graph.create(event_node)
    else:
        # El nodo ya existe, obtenemos una referencia a él
        print('El evento ya existía', net, code)
        event_node = result[0]["e"]

    return event_node
  except Exception as e:
    try:
      # El grafo está vacío, creamos el primer nodo
      print('Creando evento: ',net, code, '(', latitud, ',', longitud, ',', z,')')
      location = py2neo.data.spatial.CartesianPoint((latitud, longitud, z))
      event_node = Node("Evento",net=net, code=code, ids=ids, coordenadas=location, magnitud=mag, fecha=time, web=url, tipo=tipo, titulo=title, lugar=place, detalle=detail)
      graph.create(event_node)
      return event_node
    except Exception as e:
      print ('Error al crear evento en linea {}'.format(sys.exc_info()[-1].tb_lineno))
      raise e




# Función que crea el nodo de una ciudad si no existe y si ya existe retorna el nodo existente.
def crear_ciudad(graph, latitud, longitud, nombre):
  try:
    # Consulta si ya existe un nodo "ciudad" con las coordenadas dadas
    location = py2neo.data.spatial.CartesianPoint((latitud, longitud))
    query = "MATCH (c:Ciudad) WHERE c.coordenadas = $location RETURN c"
    result = graph.run(query, location = location).data()

    if not result:
        # El nodo no existe, así que lo creamos
        print('Creando ciudad', nombre)
        ciudad_node = Node("Ciudad", coordenadas = location, nombre = nombre)
        graph.create(ciudad_node)
    else:
        print('La ciudad ya existía', nombre)
        # El nodo ya existe, obtenemos una referencia a él
        ciudad_node = result[0]["c"]

    return ciudad_node
  except Exception as e:
    try:
      # El grafo está vacío, creamos el primer nodo
      print('Creando ciudad', nombre)
      ciudad_node = Node("Ciudad", coordenadas = location, nombre = nombre)
      graph.create(ciudad_node)
      return ciudad_node
    except Exception as e:
      print ('Error al crear ciudad en linea {}'.format(sys.exc_info()[-1].tb_lineno))
      raise e



# Función que crea el arco entre un nodo ciudad y un nodo evento, si ese arco no existe y si ya existe retorna el existente.
def crear_arco(graph, evento, ciudad, distance, direction):
    # Consulta si ya existe un arco entre ciudad y evento
    query = "MATCH (evento)-[r:NEARBY]->(ciudad) WHERE ID(evento) = $evento_id AND ID(ciudad) = $ciudad_id RETURN r"
    result = graph.run(query, evento_id=evento.identity, ciudad_id=ciudad.identity).data()

    if not result:
        # El arco no existe, así que lo creamos
        arco = Relationship(evento, "NEARBY", ciudad, distancia=distance, direccion=direction)
        graph.create(arco)
    else:
        print('Ya existía arco')
        # El nodo ya existe, obtenemos una referencia a él
        arco = result[0]["r"]

    return arco

In [ ]:
# función que obtiene todos los eventos para una zona geográfica y rango de fechas especificados
def get_earthquake_data(start_date, end_date, minlatitude, maxlatitude, minlongitude, maxlongitude):
    # url WebService de USGS
    url_usgs = "https://earthquake.usgs.gov/fdsnws/event/1/query"
    # Parameters for the API request
    params = {
        "format": "geojson",
        "starttime": start_date,
        "endtime": end_date,
        "minlatitude" : minlatitude,
        "maxlatitude" : maxlatitude,
        "minlongitude" : minlongitude,
        "maxlongitude" : maxlongitude
        #"includeallorigins": "true"
    }
    # Sending the API request and getting the response
    response = requests.get(url_usgs, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Error occurred while fetching earthquake data.")
        print(response)
        return None

def process_event(feature):
  try:
    # obtengo los datos del evento
    properties = feature["properties"]
    geometry = feature["geometry"]
    [lat, long, z] = geometry["coordinates"]
    place = properties["place"]
    net = properties["net"]
    code = properties["code"]
    mag = properties["mag"]
    time = properties["time"]
    url = properties["url"]
    detail = properties["detail"]
    ids = properties["ids"]
    tipo = properties["type"]
    title = properties["title"]

    fecha = datetime.utcfromtimestamp(time/1000)

    # creo el nodo evento
    event_node = crear_evento(graph, net, code, ids, tipo, title, place, mag, fecha, url, detail, lat, long, float(z))

    # consulto la url de detalles para obtener las ciudades cercanas
    response = requests.get(detail)
    if response.status_code == 200:
      data = response.json()
      #print(data)
      properties = data["properties"]
      prod = properties["products"]
      # si existe un producto del tipo nearby-cities, proceso la información existente, referente a las ciudades cercanas
      if("nearby-cities" in prod):
        nbcities = prod["nearby-cities"][0]
        content = nbcities["contents"]["nearby-cities.json"]
        # obtengo las cidades cercanas
        url2 = content["url"]
        response2 = requests.get(url2)
        if response2.status_code == 200:
          cities = response2.json()
          #recorro todas las ciudades de la colección
          for city in cities:
            city_name = city["name"]
            city_lat = city["latitude"]
            city_long = city["longitude"]
            city_node = crear_ciudad(graph, city_long, city_lat, city_name)
            dist= city["distance"]
            direc = city["direction"]
            crear_arco(graph, event_node, city_node, dist, direc)

        else:
          print(response2.headers)
    else:
      print(response.headers)

  except Exception as e:
    print('Error en linea {}'.format(sys.exc_info()[-1].tb_lineno))
    print( str(e) )
    print('No detail')
    return []



In [ ]:
# defino los límites de la zona geográfica a consultar
maxlatitude = 44
minlatitude = 30
maxlongitude = -114
minlongitude = -125

# ya cargamos datos desde el 24 de mayo en delante
# defino rango de fechas para la consulta
start_date = "2023-05-24"
end_date = "2023-06-25"

# Obtengo los datos de eventos en el rango de fechas especificado
earthquake_data = get_earthquake_data(start_date, end_date, minlatitude, maxlatitude, minlongitude, maxlongitude)


# Proceso los datos obtenidos
if earthquake_data:
    # Extraigo la info de los eventos
    features = earthquake_data["features"]
    # recorro y proceso los eventos obtenidos
    for feature in features:
      process_event(feature)


El evento ya existía nc 73904846
2.204940546644569
La ciudad ya existía San Martin, California
Ya existía arco
La ciudad ya existía Morgan Hill, California
Ya existía arco
La ciudad ya existía Gilroy, California
Ya existía arco
La ciudad ya existía Interlaken, California
Ya existía arco
La ciudad ya existía Corralitos, California
Ya existía arco
El evento ya existía nc 73904841
2.204940546644569
1.1069316771812328
La ciudad ya existía Daly City, California
Ya existía arco
La ciudad ya existía Colma, California
Ya existía arco
La ciudad ya existía San Francisco Zoo, California
Ya existía arco
La ciudad ya existía Pacifica, California
Ya existía arco
La ciudad ya existía South San Francisco, California
Ya existía arco
El evento ya existía ci 40494512
1.1069316771812328
La ciudad ya existía Lytle Creek, CA
Ya existía arco
La ciudad ya existía Devore, CA
Ya existía arco
La ciudad ya existía Rancho Cucamonga, CA
Ya existía arco
La ciudad ya existía Fontana, CA
Ya existía arco
La ciudad ya e

KeyboardInterrupt: ignored

In [ ]:
# función que comprueba si un evento pertenece a un cluster y lo agrega de ser así
def buscar_cluster(graph, evento):
  try:
    # parámetros para determinar si dos eventos pertenecen al mismo cluster
    alpha = 0.001 # peso de la distancia entre eventos
    beta = 0.000001 # peso del tiempo transcurrido entre eventos
    f_min = 1 # valor mínimo para considerarse cercanos
    coord = evento["coordenadas"]

    # Sentencia que trae todos los nodos que cumplen con el criterio de correlación
    sentencia = '''MATCH(c: Evento) with *,
                    point.distance(
                        point({latitude:c.coordenadas.y, longitude:c.coordenadas.x, height:c.coordenadas.z}),
                        point({latitude:$y, longitude:$x, height:$z}))/1000
                    as dist
                    with *, abs(datetime(c.fecha).epochMillis - datetime($fecha).epochMillis) as dt
                    where ID(c) > $evento_id
                    and $fmin  < 1/($alpha*dist + $beta*dt)  return c, 1/($alpha*dist + $beta*dt) as correl'''

    results = graph.run(sentencia, y=coord.y, x=coord.x, z=coord.z, fecha=evento["fecha"], fmin=f_min, alpha=alpha, beta=beta, evento_id=evento.identity).data()

    # si hay nodos correlacionados...
    if results:
      cant = len(results)
      print('Tiene', cant, 'nodos relacionados')

      # Miro si mi nodo ya está en un cluster, sino lo creo
      sentencia = '''MATCH (e: Evento)-[r]-(c: Cluster) where ID(e) = $evento_id return c'''
      clus = graph.run(sentencia, evento_id=evento.identity).data()
      if not clus:
        print('Creando cluster.')
        cluster = Node("Cluster", coordenadas = evento["coordenadas"], inicio = evento["fecha"], fin = evento["fecha"], magnitud_avg = evento["magnitud"], cant_eventos = 1)
        graph.create(cluster)
        arco = Relationship(evento, "CORRELATED", cluster)
        graph.create(arco)
      else:
          print('Cluster ya existente.')
          # El cluster ya existe, obtenemos una referencia a él
          cluster = clus[0]["c"]

      # recorro los nodos que cumplen los criterios de correlación
      i=1
      for result in results:
        print('Procesando correlacion', i, 'de', cant)
        i += 1
        evt2 = result["c"]
        correl = result["correl"]

        # me fijo si este nodo ya pertenecía a otro cluster
        sentencia = '''MATCH (e: Evento)-[r]-(c: Cluster)-[r2]-(e2: Evento) where e <> e2 and ID(e) = $evento_id and ID(c) <> $cluster_id return e2, c'''
        otro_clus = graph.run(sentencia, evento_id=evt2.identity, cluster_id = cluster.identity).data()

        # si el nodo estaba en otro cluster, busco todos los nodos de ese otro cluster y los pongo en mi nuevo cluster, lueg elimino el cluster anterior
        if otro_clus:
          print('El nodo ya estaba en otro cluster')
          otro_cluster = otro_clus[0]["c"]
          for nodos_clus in otro_clus:
            print('Cambio un nodo de cluster.')
            nodo_clus = nodos_clus["e2"]
            arco = Relationship(nodo_clus, "CORRELATED", cluster)
            graph.create(arco)
          print('Borro cluster viejo')
          graph.delete(otro_cluster)

        # Agrego el nodo al cluster
        arco = Relationship(evt2, "CORRELATED", cluster)
        graph.create(arco)
        arco = Relationship(evt2, "CORRELATION", evento, factor=correl)
        graph.create(arco)

  except Exception as e:
    print('Error en linea {}'.format(sys.exc_info()[-1].tb_lineno))
    print( str(e) )


In [ ]:
# Recorro todos los eventos e identifico clusters

sentencia = "MATCH(e: Evento) return e ORDER BY ID(e) ASC"
results = graph.run(sentencia).data()
cant = len(results)
i = 1
for evt in results:
  node = evt["e"]
  print("Procesando Evento", i, "de", cant)
  i+=1
  buscar_cluster(graph, node)

Se han truncado las últimas 5000 líneas del flujo de salida.
Procesando correlacion 3 de 3
Procesando Evento 9 de 1701
Tiene 2 nodos relacionados
Cluster ya existente.
Procesando correlacion 1 de 2
Procesando correlacion 2 de 2
Procesando Evento 10 de 1701
Tiene 1 nodos relacionados
Cluster ya existente.
Procesando correlacion 1 de 1
Procesando Evento 11 de 1701
Procesando Evento 12 de 1701
Tiene 3 nodos relacionados
Creando cluster.
Procesando correlacion 1 de 3
Procesando correlacion 2 de 3
Procesando correlacion 3 de 3
Procesando Evento 13 de 1701
Tiene 2 nodos relacionados
Cluster ya existente.
Procesando correlacion 1 de 2
Procesando correlacion 2 de 2
Procesando Evento 14 de 1701
Tiene 1 nodos relacionados
Cluster ya existente.
Procesando correlacion 1 de 1
Procesando Evento 15 de 1701
Procesando Evento 16 de 1701
Tiene 1 nodos relacionados
Creando cluster.
Procesando correlacion 1 de 1
Procesando Evento 17 de 1701
Procesando Evento 18 de 1701
Tiene 1 nodos relacionados
Creando c

In [ ]:
# Recorro todos los clusters para actualizar sus valores

sentencia = '''Match(c:Cluster)-[r]-(e:Evento)
                return c,
                count(e) as cant,
                min(e.fecha) as ini,
                max(e.fecha) as fn,
                avg(e.magnitud) as mag,
                point({x: avg(e.coordenadas.x), y: avg(e.coordenadas.y), z: avg(e.coordenadas.z)}) as cord'''
results = graph.run(sentencia).data()
cant = len(results)
i = 1
for cluster in results:
  node = cluster["c"]
  node["inicio"] = cluster["ini"]
  node["fin"] = cluster["fn"]
  node["magnitud_avg"] = cluster["mag"]
  node["coordenadas"] = cluster["cord"]
  node["cant_eventos"] = cluster["cant"]
  graph.push(node)
  print("Procesando Cluster", i, "de", cant)
  i+=1

Procesando Cluster 1 de 387
Procesando Cluster 2 de 387
Procesando Cluster 3 de 387
Procesando Cluster 4 de 387
Procesando Cluster 5 de 387
Procesando Cluster 6 de 387
Procesando Cluster 7 de 387
Procesando Cluster 8 de 387
Procesando Cluster 9 de 387
Procesando Cluster 10 de 387
Procesando Cluster 11 de 387
Procesando Cluster 12 de 387
Procesando Cluster 13 de 387
Procesando Cluster 14 de 387
Procesando Cluster 15 de 387
Procesando Cluster 16 de 387
Procesando Cluster 17 de 387
Procesando Cluster 18 de 387
Procesando Cluster 19 de 387
Procesando Cluster 20 de 387
Procesando Cluster 21 de 387
Procesando Cluster 22 de 387
Procesando Cluster 23 de 387
Procesando Cluster 24 de 387
Procesando Cluster 25 de 387
Procesando Cluster 26 de 387
Procesando Cluster 27 de 387
Procesando Cluster 28 de 387
Procesando Cluster 29 de 387
Procesando Cluster 30 de 387
Procesando Cluster 31 de 387
Procesando Cluster 32 de 387
Procesando Cluster 33 de 387
Procesando Cluster 34 de 387
Procesando Cluster 35 d